# <b><span style='color:#0B2F9F'>Pengantar</span></b>

Dalam pengelolaan data, kemampuan untuk menampilkan informasi dengan format yang tepat serta menggali data secara mendalam menjadi hal yang sangat penting. Format angka dan tanggal membantu memastikan data tersaji secara konsisten, mudah dibaca, dan sesuai konteks, sehingga hasil analisis lebih akurat dan informatif. Sementara itu, penggunaan subquery memungkinkan pengambilan data secara lebih fleksibel dan efisien, dengan cara memecah proses pencarian menjadi langkah-langkah yang lebih terarah. Dengan memahami kedua konsep ini, seseorang dapat mengolah data dengan lebih rapi, melakukan analisis yang lebih mendalam, serta menghasilkan informasi yang lebih bermakna dan dapat diandalkan.


# <b><span style='color:#0B2F9F'>Set up</span></b>

Dibutuhkan proses autentikasi dari Google Colab ke Google Big Query. Ikuti langkah berikut https://drive.google.com/file/d/1gW8alZ_PrvcrsieqWCHOR4ssLI_25BRc/view untuk detail step-by-step nya

In [ ]:
# Import library yang dibutuhkan
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Buat BigQuery client
project_id = 'dqlab-462502'
client = bigquery.Client(project = project_id)

Sebagai Data Engineer, Antara (Jr. Data Engineer di DQPizza) ditugaskan untuk membuat data mart untuk kebutuhan detail transaksi. Dengan memanfaatkan relasi antar table, Antara mengeksekusi suatu perintah  untuk membuat table f_detail_transaksi (sebuah tabel fact yang berisi data transaksi secara rinci, mencakup ukuran-ukuran kuantitatif seperti jumlah, harga, total nilai transaksi, dan lain sebagainya) hingga terbentuklah tabel f_detail_transaksi.

<br>
<code>SELECT * FROM dqlab-9876543.dq_pizza.f_detail_transaksi</code>
<br><br>
<img src="https://raw.githubusercontent.com/bachtiyarma/Material/main/Image/Materi-SQL/SQL%20-%20f_detail_transaksi.png" width="30%">

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

Executing query with job ID: f6068bd3-753b-4105-be5b-5eb6f7f45860
Query executing: 0.44s
Job ID f6068bd3-753b-4105-be5b-5eb6f7f45860 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail
0,TRX019321,TRXD043959,CUST003319,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-11-26,13:58:49,14:26:52,0,None
1,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-01-12,20:00:30,20:27:49,0,None
2,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-03-10,19:10:04,19:39:33,0,None
3,TRX013010,TRXD029448,CUST005604,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-08-05,18:06:43,18:32:51,0,None
4,TRX008857,TRXD020142,CUST004778,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-05-29,13:21:21,13:45:23,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48614,TRX003266,TRXD007413,CUST007085,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-02-24,18:40:58,19:01:17,0,None
48615,TRX008441,TRXD019204,CUST007254,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-05-21,21:14:52,21:30:47,0,None
48616,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-01-27,16:28:38,16:54:32,0,None
48617,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-10-10,17:54:55,18:06:01,0,None


In [ ]:
%%bigquery --project {project_id} --verbose

CREATE OR REPLACE TABLE dqlab-462502.dq_pizza.f_detail_transaksi AS
SELECT
    t1.order_id,
    t1.order_details_id,
    t2.customer_id,
    t2.order_maker_id,
    t4.name AS pizza_name,
    t4.category,
    t3.size,
    CAST(REPLACE(t3.price, 'IDR', '') as float64) as price,
    CAST(REPLACE(t3.production_cost, 'IDR', '') as float64) as production_cost,
    t1.quantity
FROM
    `dqlab-9876543.dq_pizza.order_details` AS t1
INNER JOIN
    `dqlab-9876543.dq_pizza.orders` AS t2
    ON t1.order_id = t2.order_id
INNER JOIN
    `dqlab-9876543.dq_pizza.pizzas` AS t3
    ON t1.pizza_id = t3.pizza_id
INNER JOIN
    `dqlab-9876543.dq_pizza.pizza_types` AS t4
    ON t3.pizza_type_id = t4.pizza_type_id;

Executing query with job ID: 1840d7fe-7854-445f-844f-77ab780f6df8
Query executing: 2.59s
Job ID 1840d7fe-7854-445f-844f-77ab780f6df8 successfully executed


Query is running:   0%|          |

""


Data dibawah (f_detail_transaksi) akan digunakan pada modul ini

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *
FROM `dqlab-462502.dq_pizza.f_detail_transaksi`

Executing query with job ID: 539472e2-4488-42d1-8062-2ab679db493d
Query executing: 1.36s
Job ID 539472e2-4488-42d1-8062-2ab679db493d successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity
0,TRX005246,TRXD011942,CUST005429,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1
1,TRX008021,TRXD018266,CUST009740,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1
2,TRX004761,TRXD010874,CUST007768,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1
3,TRX007433,TRXD016895,CUST007656,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1
4,TRX018254,TRXD041472,CUST005109,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1
...,...,...,...,...,...,...,...,...,...,...
48618,TRX017862,TRXD040544,CUST009381,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1
48619,TRX001622,TRXD003654,CUST001704,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1
48620,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1
48621,TRX002813,TRXD006366,CUST006714,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1


# <b>A. <span style='color:#0B2F9F'><code>DATA TYPE</code></span></b>

Data type dalam BigQuery adalah cara untuk mendefinisikan jenis data yang akan disimpan dalam tabel. Setiap kolom dalam tabel memiliki tipe data tertentu, yang menentukan apa yang dapat disimpan di kolom tersebut dan bagaimana data tersebut dapat diproses. Memahami tipe data ini penting untuk desain skema yang efisien dan untuk memastikan bahwa kueri dapat dijalankan dengan optimal.

<br>
<img src="https://hackolade.com/help/lib/BigQuery%20data%20types%20table.png" width="40%">

docs : <i><a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types">https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types</a></i>

# <b>B. <span style='color:#0B2F9F'><code>Operasi Tipe Data Numerik</code></span></b>

Dalam SQL, tipe data numerik digunakan untuk menyimpan nilai angka dan dapat dikelompokkan menjadi beberapa jenis sesuai peruntukannya (INT, FLOAT, NUMERIC, dll). Selain itu pada tipe data Numerik juga dapat dilakukan operasi matematis diantaranya :
<ol>
    <li>Operasi Aritmatika</li>
    <li>Fungsi Matematis</li>
    <li>Agregasi</li>
</ol>

## <b>B.1. <span style='color:#0B2F9F'><code>Operasi Aritmatika</code></span></b>
SQL mendukung operasi dasar aritmatika seperti penjumlahan, pengurangan, perkalian, pembagian dan modulo.

<img src="https://mazamanbd.wordpress.com/wp-content/uploads/2014/09/dd.jpg"><br><br>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL%20-%20OperasiAritmatika.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/operators">https://cloud.google.com/bigquery/docs/reference/standard-sql/operators</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel f_detail_transaksi, tampilkan semua kolom dan tambahkan satu kolom 'total_price' yang merupakan hasil perkalian antara harga dengan kuantitas (terbeli)! Urutkan berdasarkan kuantitas paling besar hingga ke terkecil </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *,
  price * quantity AS total_price
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
ORDER BY quantity DESC

Executing query with job ID: fcb83389-faa2-4664-8e93-0b0caf6315a9
Query executing: 0.38s
Job ID fcb83389-faa2-4664-8e93-0b0caf6315a9 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,2024-03-30,13:24:35,13:35:43,0,None,388800.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,2024-05-19,12:42:46,13:11:40,0,None,388800.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,2024-09-19,14:16:35,14:40:53,0,None,672300.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,2024-11-18,12:25:12,12:50:38,0,None,407025.0
4,TRX000394,TRXD000903,CUST006034,EMP000231,THE PEPPERONI PIZZA,CLASSIC,M,101250.0,74682.0,3,2024-01-07,14:02:18,14:31:55,0,None,303750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48614,TRX003266,TRXD007413,CUST007085,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-02-24,18:40:58,19:01:17,0,None,191565.0
48615,TRX008441,TRXD019204,CUST007254,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-05-21,21:14:52,21:30:47,0,None,191565.0
48616,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-01-27,16:28:38,16:54:32,0,None,291195.0
48617,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-10-10,17:54:55,18:06:01,0,None,291195.0


#### <i><b><span style='color:#55679C'>Quest</span> : Selain total price, tambahkan juga total profit yang didapat tiap transaksi! (Lanjutkan hasil query quest diatas) </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *,
  price * quantity AS total_price,
  (price - production_cost) * quantity AS total_profit
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
ORDER BY quantity DESC

Executing query with job ID: 5967ae56-e3c4-4c46-b5ba-f1e2d287040b
Query executing: 0.35s
Job ID 5967ae56-e3c4-4c46-b5ba-f1e2d287040b successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price,total_profit
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,2024-03-30,13:24:35,13:35:43,0,None,388800.0,62532.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,2024-05-19,12:42:46,13:11:40,0,None,388800.0,62532.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,2024-09-19,14:16:35,14:40:53,0,None,672300.0,196344.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,2024-11-18,12:25:12,12:50:38,0,None,407025.0,108621.0
4,TRX000394,TRXD000903,CUST006034,EMP000231,THE PEPPERONI PIZZA,CLASSIC,M,101250.0,74682.0,3,2024-01-07,14:02:18,14:31:55,0,None,303750.0,79704.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48614,TRX003266,TRXD007413,CUST007085,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-02-24,18:40:58,19:01:17,0,None,191565.0,21060.0
48615,TRX008441,TRXD019204,CUST007254,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-05-21,21:14:52,21:30:47,0,None,191565.0,21060.0
48616,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-01-27,16:28:38,16:54:32,0,None,291195.0,30457.0
48617,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-10-10,17:54:55,18:06:01,0,None,291195.0,30457.0


#### <i><b><span style='color:#55679C'>Quest</span> : Bagaimana dengan customer paling loyal? Paling loyal diukur dari banyak profit restaurant yang dihasilkan. Tampilkan customer_id, total_frequency (berapa kali pembelian dalam setahun) dan total_profit yang diperoleh perusahaan, kelompokkan tiap customer_id! Urutkan berdasarkan customer_id dengan profit paling tinggi ke paling rendah! </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  COUNT(DISTINCT order_id) AS total_frequency,
  SUM((price - production_cost) * quantity) AS total_profit
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
GROUP BY customer_id
ORDER BY 3 DESC

Executing query with job ID: bc42e9a0-e558-434c-87f9-dfd6fdf8897d
Query executing: 0.54s
Job ID bc42e9a0-e558-434c-87f9-dfd6fdf8897d successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,total_frequency,total_profit
0,CUST006333,741,47622169.0
1,CUST009381,702,44017426.0
2,CUST007656,614,37358334.0
3,CUST003549,580,36914292.0
4,CUST006964,521,32351562.0
...,...,...,...
115,CUST008078,21,1070496.0
116,CUST005536,24,987714.0
117,CUST006892,12,858681.0
118,CUST003871,13,685908.0


## <b>B.2. <span style='color:#0B2F9F'><code>Fungsi Matematis</code></span></b>
SQL juga mendukung fungsi matematis dasar yang disajikan pada tabel berikut :

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>ABS()</code></td>
            <td>Mengambil nilai absolut</td>
        </tr>
        <tr>
            <td><code>ROUND()</code></td>
            <td>Membulatkan angka</td>
        </tr>
        <tr>
            <td><code>CEIL()</code></td>
            <td>Membulatkan ke atas</td>
        </tr>
        <tr>
            <td><code>FLOOR()</code></td>
            <td>Membulatkan ke bawah</td>
        </tr>
        <tr>
            <td><code>POW()</code></td>
            <td>Menghitung pangkat</td>
        </tr>
    </tbody>
</table>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL-FungsiMatematis.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Sendja (Data Analyst di DQPizza) baru menyadari bahwa profit yang ia hitung pada quest sebelumnya merupakan profit sebelum pajak. Asumsikan pajak yang perlu dikeluarkan adalah 11% tiap kali transaksi. Hitung profit_before_tax, profit_after_tax dan tax_paid tiap order_id dan customer_id! Lakukan perhitungan untuk setiap transaksi yang valid (order_details_id tidak kosong)!</b></i>

Note : Bulatkan profit after tax sebanyak 2 angka dibelakang koma agar lebih rapi

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  order_id,
  customer_id,
  SUM((price - production_cost) * quantity) AS profit_before_tax,
  ROUND(0.89 * SUM((price - production_cost) * quantity), 2) AS profit_after_tax,
  ROUND(0.11 * SUM((price - production_cost) * quantity), 2) AS tax_paid
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
WHERE order_details_id IS NOT NULL
GROUP BY order_id, customer_id
ORDER BY 3 DESC

Executing query with job ID: b02e626c-aee0-4cf8-9e43-1adb979441b7
Query executing: 0.32s
Job ID b02e626c-aee0-4cf8-9e43-1adb979441b7 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,customer_id,profit_before_tax,profit_after_tax,tax_paid
0,TRX018845,CUST007150,747954.0,665679.06,82274.94
1,TRX010760,CUST001704,684855.0,609520.95,75334.05
2,TRX004482,CUST003570,518481.0,461448.09,57032.91
3,TRX006169,CUST007656,494586.0,440181.54,54404.46
4,TRX009331,CUST004537,483813.0,430593.57,53219.43
...,...,...,...,...,...
21341,TRX014887,CUST002604,10854.0,9660.06,1193.94
21342,TRX003839,CUST005127,10854.0,9660.06,1193.94
21343,TRX015013,CUST003570,10854.0,9660.06,1193.94
21344,TRX013817,CUST004547,10854.0,9660.06,1193.94


# <b>C. <span style='color:#0B2F9F'><code>Operasi Tipe Data Date</code></span></b>

Dalam SQL, tipe data date digunakan untuk menyimpan atau mengoperasikan data tanggal. Tipe data tanggal dapat dilakukan operasi berikut, diantaranya :
<ol>
    <li>Menampilkan Tanggal & Waktu saat ini</li>
    <li>Menambahkan atau Mengurangi Tanggal</li>
    <li>Mendapatkan Bagian Tanggal</li>
    <li>Menghitung Selisih Tanggal</li>
</ol>

<br><b>NOTE</b><br><br>
Hal-hal yang perlu diperhatikan dalam penggunaan operasi `date` adalah :
<ul>
    <li>Format tanggal standar SQL adalah 'YYYY-MM-DD' atau tahun - bulan - tanggal, contoh : 2024-10-14 menunjukan tanggal 14 October 2024</li>
    <li>Mengakses tanggal pada SQL perlu menggunakan tanda apit petik satu, contoh : '2024-10-14'</li>
    <li>Pastikan nilai atau kolom tanggal tipe datanya adalah DATE atau TIMESTAMP, jika tidak maka bisa dikonversi menjadi <code>DATE(tanggal)</code> atau <code>TIMESTAMP(tanggal)</code></li>
</ul>

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions</a></i>

## <b>C.1. <span style='color:#0B2F9F'><code>Menampilkan Tanggal & Waktu Saat Ini</code></span></b>

Untuk menampilkan tanggal dan waktu saat ini dapat menggunakan fungsi berikut
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>CURRENT_DATE()</code></td>
            <td>Mendapatkan Tanggal Hari ini Saja</td>
        </tr>
        <tr>
            <td><code>CURRENT_TIMESTAMP()</code></td>
            <td>Mendapatkan Tanggal & Waktu Hari ini (UTC)</td>
        </tr>
    </tbody>
</table>

UTC atau (Universal Time Coordinated) adalah waktu dasar yang tidak terpengaruh oleh zona waktu dan <i>daylight saving time</i>. UTC digunakan sebagai acuan untuk menghitung waktu di berbagai zona waktu di seluruh dunia (standar Greenwich Mean Time (London)). Selengkapnya baca <a href="https://internasional.kompas.com/read/2021/03/04/192042570/sejarah-penetapan-zona-waktu-di-dunia-hingga-usulan-penghapusannya?page=all">disini</a>.

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan semua kolom pada tabel f_detail_transaksi dan tambahkan kolom data_collection_time sebagai kolom metadata waktu pengambilan data</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *,
  CURRENT_DATETIME('Asia/Jakarta') AS data_collection_time
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

Executing query with job ID: 52d3acbc-4b66-4f2e-9fdc-1e68f4329df3
Query executing: 1.49s
Job ID 52d3acbc-4b66-4f2e-9fdc-1e68f4329df3 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,data_collection_time
0,TRX019321,TRXD043959,CUST003319,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-11-26,13:58:49,14:26:52,0,None,2025-11-11 10:13:50.055702
1,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-01-12,20:00:30,20:27:49,0,None,2025-11-11 10:13:50.055702
2,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-03-10,19:10:04,19:39:33,0,None,2025-11-11 10:13:50.055702
3,TRX013010,TRXD029448,CUST005604,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-08-05,18:06:43,18:32:51,0,None,2025-11-11 10:13:50.055702
4,TRX008857,TRXD020142,CUST004778,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,2024-05-29,13:21:21,13:45:23,0,None,2025-11-11 10:13:50.055702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48614,TRX003266,TRXD007413,CUST007085,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-02-24,18:40:58,19:01:17,0,None,2025-11-11 10:13:50.055702
48615,TRX008441,TRXD019204,CUST007254,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,2024-05-21,21:14:52,21:30:47,0,None,2025-11-11 10:13:50.055702
48616,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-01-27,16:28:38,16:54:32,0,None,2025-11-11 10:13:50.055702
48617,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,2024-10-10,17:54:55,18:06:01,0,None,2025-11-11 10:13:50.055702


## <b>C.2. <span style='color:#0B2F9F'><code>Mengubah String menjadi Tanggal</code></span></b>

Adakalanya format tanggal yang disimpan pada database tidak disimpan dalam format date yang baku (<a href="https://www.iso.org/iso-8601-date-and-time-format.html">ISO 8601</a>) sehingga perlu penyesuaian. Gunakan : <br><br>
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>PARSE_DATE(format, string_expression)</code></td>
            <td>Mengubah String menjadi Date</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE(format, date_expression)</code></td>
            <td>Mengubah Date menjadi String</td>
        </tr>
    </tbody>
</table><br>

Untuk detail format tanggal akses link berikut https://docs.cloud.google.com/bigquery/docs/reference/standard-sql/format-elements

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan order_date dan order_date_iso pada tabel f_detail_transaksi. Dimana kolom order_date merupakan kolom dengan data asli dan order_date_iso merupakan kolom order_date yang diubah formatnya menjadi YYYY-MM-DD sesuai ISO 8601!</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  order_date,
  PARSE_DATE('%m/%d/%Y', order_date) AS order_date_iso
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

Executing query with job ID: 1d2a0648-0e79-428b-bbb3-bd3177ff5d1b
Query executing: 0.29s


ERROR:
 400 No matching signature for function PARSE_DATE
  Argument types: STRING, DATE
  Signature: PARSE_DATE(STRING, STRING)
    Argument 2: Unable to coerce type DATE to expected type STRING at [3:3]; reason: invalidQuery, location: query, message: No matching signature for function PARSE_DATE
  Argument types: STRING, DATE
  Signature: PARSE_DATE(STRING, STRING)
    Argument 2: Unable to coerce type DATE to expected type STRING at [3:3]

Location: US
Job ID: 1d2a0648-0e79-428b-bbb3-bd3177ff5d1b



## <b>C.3. <span style='color:#0B2F9F'><code>Menambahkan atau Mengurangi Tanggal</code></span></b>

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>DATE_ADD(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Menambahkan Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_SUB(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Mengurangi Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_DIFF(end_date, start_date, date_part)</code></td>
            <td>Menghitung selisih tanggal</td>
        </tr>
    </tbody>
</table><br>

Sebagai contoh misalkan hari ini tanggal 30 Oktober 2025. Ingin diperoleh tanggal tepat 3 bulan yang lalu maka dapat digunakan sintaks `DATE_SUB(DATE('2025-10-30'), INTERVAL 3 MONTH)` maka akan diperoleh tanggal 30 Juli 2025.



#### <i><b><span style='color:#55679C'>Quest</span> : Manajer ingin melakukan analisa demografi customer, pada tabel customers tampilkan customer_id, customer_name, gender dan age-nya per hari ini!</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  customer_name,
  gender,
  DATE_DIFF(CURRENT_DATE(), DATE(birth_date), YEAR) AS age
FROM `dqlab-9876543.dq_pizza.customers`

Executing query with job ID: 6bf9b793-013b-4e27-985b-17401de16844
Query executing: 0.82s
Job ID 6bf9b793-013b-4e27-985b-17401de16844 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_name,gender,age
0,CUST007150,Salsabila Purnawati,F,35
1,CUST003319,Cornelia Wijayanti,F,28
2,CUST007254,Elma Maryati,F,41
3,CUST004392,Najwa Natsir,F,45
4,CUST005127,Hani Palastri,F,22
...,...,...,...,...
132,CUST009056,Gada Winarsih,M,25
133,CUST009366,Mulyanto Suryono,M,24
134,CUST009522,Cakrawangsa Hutagalung,M,36
135,CUST009700,Asirwanda Hidayat,M,34


## <b>C.5. <span style='color:#0B2F9F'><code>Mendapatkan Bagian pada Tanggal</code></span></b>

Mengambil bagian tertentu dari tipe data tanggal, waktu, atau timestamp terkadang sangat perlu dilakukan untuk analisa data. Jika ingin didapatkan elemen tertentu, seperti tahun, bulan, hari, atau jam, dari nilai tanggal atau waktu gunakan fungsi berikut<br><br>

<code>EXTRACT(bagian FROM tanggal)</code>
<br><br>

Bagian yang bisa diambil pada tanggal diantaranya adalah
<table border="1">
    <thead>
        <tr>
            <th>Bagian</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>YEAR</code></td>
            <td>Mendapatkan Tahun pada Tanggal</td>
        </tr>
        <tr>
            <td><code>MONTH</code></td>
            <td>Mendapatkan Bulan pada Tanggal</td>
        </tr>
        <tr>
            <td><code>DAY</code></td>
            <td>Mendapatkan Hari (1-31) pada Tanggal</td>
        </tr>
        <tr>
            <td><code>WEEK</code></td>
            <td>Mendapatkan Minggu ke- pada Tanggal</td>
        </tr>
         <tr>
            <td><code>QUARTER</code></td>
            <td>Mendapatkan Kuartal pada Tanggal</td>
        </tr>
        <tr>
            <td><code>HOUR</code></td>
            <td>Mendapatkan Jam pada Waktu</td>
        </tr>
        <tr>
            <td><code>MINUTE</code></td>
            <td>Mendapatkan Menit pada Waktu</td>
        </tr>
        <tr>
            <td><code>SECOND</code></td>
            <td>Mendapatkan Detik pada Waktu</td>
        </tr>
    </tbody>
</table>
<br>

Namun jika menginginkan bagian yang lebih detail seperti Nama Bulan atau Nama Hari pada tanggal tertentu atau menampilkan format tanggal sesuai dengan keinginan dapat menggunakan sintaks `FORMAT_DATE(format_element, tanggal)`. <br><br>

<table border="1">
    <thead>
        <tr>
            <th>Sintaks</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>FORMAT_DATE('%A', tanggal)</code></td>
            <td>Mendapatkan Nama Hari pada Tanggal Tertentu</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE('%B', tanggal)</code></td>
            <td>Mendapatkan Nama Bulan pada Tanggal Tertentu</td>
        </tr>
    </tbody>
</table>
<br>

Detail format_element dapat diakses dilaman <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/format-elements#format_elements_date_time"> berikut</a>.





#### <i><b><span style='color:#55679C'>Quest</span> : Hitung total transaksi dan total pendapatan yang diperoleh tiap bulan pada tahun 2024 di DQPizza! </b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  EXTRACT(YEAR FROM PARSE_DATE('%m/%d/%Y', order_date)) AS year,
  EXTRACT(MONTH FROM PARSE_DATE('%m/%d/%Y', order_date)) AS month,
  FORMAT_DATE('%B', PARSE_DATE('%m/%d/%Y', order_date)) AS month_name,
  COUNT(DISTINCT order_id) AS total_transaction,
  SUM(price * quantity) AS total_revenue
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
GROUP BY 1, 2, 3
ORDER BY 1, 2

Executing query with job ID: e903f6be-894c-4f2d-83e5-78b819d964a0
Query executing: 0.33s


ERROR:
 400 No matching signature for function PARSE_DATE
  Argument types: STRING, DATE
  Signature: PARSE_DATE(STRING, STRING)
    Argument 2: Unable to coerce type DATE to expected type STRING at [2:21]; reason: invalidQuery, location: query, message: No matching signature for function PARSE_DATE
  Argument types: STRING, DATE
  Signature: PARSE_DATE(STRING, STRING)
    Argument 2: Unable to coerce type DATE to expected type STRING at [2:21]

Location: US
Job ID: e903f6be-894c-4f2d-83e5-78b819d964a0



# <b>D. <span style='color:#0B2F9F'><code>SUB-QUERY</code></span></b>
Subquery adalah sebuah query (pernyataan SQL) yang terletak di dalam query lain. Subquery dapat digunakan untuk mengambil data yang kemudian akan digunakan oleh query utama.<br><br>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, hitung rata-rata umur customer dan bulatkan ke dua tempat desimal!</b></i>

In [ ]:
%%bigquery df --project {project_id} --verbose

SELECT
  ROUND(AVG(DATE_DIFF(CURRENT_DATE(), DATE(birth_date), YEAR)), 2) AS age
FROM `dqlab-9876543.dq_pizza.customers`

Executing query with job ID: 8b4db1e3-0668-4f87-91d5-20b0af14d2d5
Query executing: 0.53s
Job ID 8b4db1e3-0668-4f87-91d5-20b0af14d2d5 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, tampilkan customer_id, age dan gendernya dimana umur customer tersebut lebih dari rata-rata umur customer secara keseluruhan!</b></i>

In [ ]:
%%bigquery df --project {project_id} --verbose

SELECT
  customer_id,
  DATE_DIFF(CURRENT_DATE(), DATE(birth_date), YEAR) AS age,
  gender
FROM `dqlab-9876543.dq_pizza.customers`
WHERE DATE_DIFF(CURRENT_DATE(), DATE(birth_date), YEAR) > (
  SELECT AVG(DATE_DIFF(CURRENT_DATE(), DATE(birth_date), YEAR)) AS age
  FROM `dqlab-9876543.dq_pizza.customers`
)

Executing query with job ID: 2152eda6-6913-4646-a9ef-efbeb942054d
Query executing: 1.08s
Job ID 2152eda6-6913-4646-a9ef-efbeb942054d successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

# <b>E. <span style='color:#0B2F9F'><code>Common Table Expression (CTE)</code></span></b>
CTE, atau Common Table Expression, adalah fitur dalam SQL yang memungkinkan Anda untuk mendefinisikan sebuah query sementara yang dapat digunakan dalam query lain. CTE membantu dalam membuat query lebih mudah dibaca dan dipelihara, serta memungkinkan penggunaan rekursi dalam query.

<img src="https://learnsql.com/blog/sql-common-table-expression-guide/cte_syntax_example.webp" width="50%">

#### <i><b><span style='color:#55679C'>Quest</span> : Siapakah customer yang paling banyak spending money di DQPizza?</b></i>

In [ ]:
%%bigquery df --project {project_id} --verbose

WITH revenue_per_cust AS (
  SELECT
    customer_id,
    SUM(price * quantity) AS total_revenue
  FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
  WHERE order_details_id IS NOT NULL
  GROUP BY customer_id
)
  SELECT * FROM revenue_per_cust
  WHERE total_revenue = (SELECT MAX(total_revenue) FROM revenue_per_cust)

Executing query with job ID: 1930f062-487f-4183-b752-0492e6a30350
Query executing: 0.33s
Job ID 1930f062-487f-4183-b752-0492e6a30350 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True \
  --to notebook \
  --output cleared_notebook.ipynb \
"/content/sample_data/tes.ipynb"

[NbConvertApp] WARNING | pattern '/content/sample_data/tes.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
